# Проект - Телеграм-бот с доступом к данным в БД

От любой базы данных нет никакого толку, если она не используется в каком-то сервисе. Чаще всего в роли сервиса выступает какое-либо веб-приложение. При этом веб-приложение - это необязательно сайт. Им может быть любая связка "интерфейс пользователя" - "сервер", например, приложение для смартфона или телеграм-бот. В этом задании мы научимся создавать телеграм-ботов, которые будут подключаться к нашей базе данных.

## Постановка задачи

Наш проект можно разделить на 4 большие части:
1. Сервер, который будет размещать ссылки на файлы со статистикой
2. Модуль для доступа к БД: описание моделей данных и api взаимодействия с этими моделями
3. Модуль взаимодействия с Телеграмом
4. Сама база данных.

Архитектура проекта будет следующей. Структура файлов:

    bot
    |- __init__.py
    |- app.py
    |- telegram.py
    |- database
       |- __init__.py
       |- dbapi.py
       |- models.py

Таким образом, наш проект будет включать три независимых модуля. Модуль сервера и модуль телеграма будут использовать функционал модуля БД в своем коде, поэтому договоритесь о протоколе взаимодействия этих модулей заранее.

### Напоминание, зачем нужен файл `__init__.py`

В уроке про модули мы с вами узнали, что любой файл является модулем, который можно импортировать. Например, при такой структуре файлов:

    |- a.py
    |- b.py
    
в файле b.py допустим следующим импорт: `from a import *`, поскольку эти файлы лежат в одной папке. Если же файл a.py лежит в подпапке, т.е. вот так:

    |- a_dir
    |  |- a.py
    |
    |- b.py
    
то логично предположить, что код из него в b.py можно импортировать командой `from a_dir.a import *`, но не тут-то было. Дело в том, что папка в питоне является модулем, если в ней есть файл с названием `__init__.py`, иначе мы не можем взаимодействовать с кодом в этой папке из других модулей. Больше того, код, описанный в этом файле, будет доступен файлов, соседствующих с папкой, через команду `from a_dir import *`, и именно такой формат импорта мы будем считать самым удобным для пользователей нашего модуля.

## Какой бот будем писать?

Напишем бот для обмена книгами. Он должен будет иметь следующий функционал:
- добавление книги в общую библиотеку
- взятие книги из библиотеки для чтения
- возвращение книги в библиотеку после чтения
- удаление книги из библиотеки (например, из-за износа)
- формирование файла со статистикой использования книги и отправление ссылки на скачивание этого файла

---

Теперь опишем подробнее, что должно быть реализовано внутри каждого из этих файлов.

# 1. Структура базы данных и моделей ORM

### `(файл database/dbapi.py)`

Для того, чтобы реализовать весь заявленный функционал, нам понадобятся 2 таблицы в Postgres: справочник всех добавленных книг и таблица, содержащая информацию о датах использования книг.

## Спецификация БД

### Таблица Books

- `book_id` - идентификатор записи в таблице
- `title` - название книги
- `author` - автор книги
- `published` - год издания книги
- `date_added` - дата, когда книга была зарегистрирована в нашей библиотеке
- `date_deleted` - дата, когда было зарегистрировано, что эту книгу больше нельзя использовать (nullable)

### Таблица Borrows

- `borrow_id` - идентификатор записи в таблице
- `book_id` - foreign key на идентификатор книги в таблице Books
- `date_start` - дата и время начала аренды книги
- `date_end` - дата и время конца аренды книги (nullable)
- `user_id` - идентификатор чата пользователя, который взял книгу

Классы моделей данных назовите соответственно `Book` и `Borrow`.

# 2. Структура кода

## 2.1. `database/dbapi.py`

В этом файле опишем класс DatabaseConnector. Он должен содержать методы, которые позволят проводить операции с книгами:

- `add` - добавляет книгу в таблицу Books. Возвращает book_id новой книги или False, если не получилось добавить.
- `delete` - помечает книгу более непригодной к использованию возвращает true/false: успешно или неуспешно прошла операция. Книгу нельзя удалить, если она у кого-то на руках.
- `list_books` - возвращает список всех добавленных в БД книг
- `get_book` - поиск книги по названию и автору. Возвращает book_id или None, если такой книги нет. Помните, что пользователь может вводить информацию в разных регистрах.
- `borrow` - добавляет новую запись в таблицу Borrows со временем начала аренды книги. Если книга уже в аренде, то ее не должно быть можно арендовать. Если у человека уже есть книга в аренде, вторую он взять не может. Возвращает borrow_id или False, если не получилось арендовать книгу.
- `get_borrow` - возвращает borrow_id брони, однозначно связанной с пользователем
- `retrieve` - изменяет запись в таблице Borrows, добавляя к ней дату возврата книги

В атрибутах класса должны содержаться параметры подключения к БД. Соединение с БД должно устанавливаться отдельно для каждой операции.

## 2.2. `app.py`

Непосредственно с сервером у нас будет связана только малая часть проекта, поскольку для работы самого бота он нам не нужен. Сервер мы будем использовать для получения статистик по использованию книги.

- URL `/download/<book_id>`: по этому адресу пользователь должен иметь возможность скачать экселевский файл со статистикой использования книги. По сути, нужно выгрузить из таблицы Borrows все записи, связанные с книгой, кроме user_id. Для того, чтобы создать такой файл, рекомендуется пользоваться функцией `read_sql` из библиотеки `pandas`. Этот файл нужно сохранить в отдельную папку, содержимое которой сервер сможет выдать пользователю, или использовать библиотеку tempfile. По этому адресу пользователь должен скачать непосредственно сам получившийся файл, после чего его можно удалить из файловой системы.

## 2.3. `telegram.py`

В этом файле мы опишем код самого телеграм-бота. В своих функциях он должен создавать объект коннектора к БД и использовать его методы. Должны быть реализованы следующие сценарии:

    /start
    /add
        - Введите название книги:
        - Название книги
        - Введите автора:
        - Автор
        - Введите год издания:
        - 2000
        - Книга добавлена (id) / Ошибка при добавлении книги
    
    /delete
        - Введите название книги:
        - Название книги
        - Введите автора:
        - Автор
        - Введите год издания:
        - 2000
        - Найдена книга: Название книги Автор 2000. Удаляем?
        - Да / Нет
        - Книга удалена / Невозможно удалить книгу
        
    /list
        выдается список книг в виде:
        Название книги, Автор, 2000;
        Название книги 2, Автор 2, 2001 (удалена);
        
    /find
        - Введите название книги:
        - Название книги
        - Введите автора:
        - Автор
        - Введите год издания:
        - 2000
        - Найдена книга: Название книги Автор 2000 / Такой книги у нас нет
        
    /borrow
        - Введите название книги:
        - Название книги
        - Введите автора:
        - Автор
        - Введите год издания:
        - 2000
        - Найдена книга: Название книги Автор 2000. Берем?
        - Да / Нет
        - Вы взяли книгу / Книгу сейчас невозможно взять
        
    /retrieve
        Вы вернули книгу Название книги Автор 2000
        
    /stats
        - Введите название книги:
        - Название книги
        - Введите автора:
        - Автор
        - Введите год издания:
        - 2000
        - Статистика доступна по адресу http://localhost/download/1/ / Нет такой книги
    
Если перейти по адресу, который указывается по ссылке, то скачивается эксель файл, о котором написано в разделе 2.2.

# 3. Создание телеграм-бота

## 3.1. Регистрация бота

Прежде, чем начать реализацию функционала телеграм-бота, его нужно создать на стороне Телеграма. Для этого есть бот BotFather, который помогает регистрировать новые боты: https://t.me/BotFather
Зарегистрируйте нового бота, введя те данные, которые он просит.

## 3.2. Установка библиотек

Мы с вами уже знакомы с сервером Flask: это программа, которая в бесконечном цикле слушает входящие запросы. Телеграм-бот можно сделать с использованием противоположного алгоритма. Мы будем в бесконечном цикле отправлять Телеграму запросы, чтобы узнать, не появилось ли что-то новое в переписках нашего бота. Для этого нам понадобится установить дополнительную библиотеку:

In [ ]:
!pip install pytelegrambotapi

Далее с использованием этой библиотеки мы можем создать объект телеграм-бота, который будет ежемоментно отправлять запросы о новых сообщениях на сервер Телеграма.

In [10]:
import telebot

token = ""  # заполните значением вашего токена, полученного от BotFather
bot = telebot.TeleBot(token)

## 3.3. Обработка сообщений

После того, как установили библиотеку и создали объект бота, мы можем начать обрабатывать сообщения. Делается это в функциях, обернутых в декоратор из библиотеки telebot. При этом в аргументах декоратора мы описываем, какие типы сообщений обрабатываются этой функцией, но без конкретных указаний, на какие команды реагирует эта функция. Похоже на то, как мы регистрировали адреса во Фласке, не правда ли?

In [4]:
@bot.message_handler(content_types=['text'])
def handle_text_message(message):
    # а вот обработку конкретных команд мы прописываем в теле функции
    if message.text == "/start":
        bot.send_message(message.from_user.id, "Добро пожаловать в чат бота-библиотеки!")

На самом деле, такой подход к ветвлению логики обработки команд бота уже устарел, и сейчас отреагировать только на команду /start можно совсем просто - прямо как во Фласке!

In [11]:
@bot.message_handler(commands=['start'])  # указали команду, при которой запускается функция
def handle_start(message):
    bot.reply_to(message, "Добро пожаловать в чат бота-библиотеки!")

Описали функционал - осталось только запустить бота! Теперь это полностью похоже на Фласк.

In [ ]:
bot.infinity_polling()

Внутри функций, описывающих обработку сообщений, будем создавать коннектор к БД и использовать его для получения нужной информации.

# 4. Запуск проекта

У нас по структуре проекта есть две точки входа: телеграм-бот и сервер - которые должны работать параллельно друг с другом. Поэтому, чтобы не мучиться с субпроцессами и прочим асинхронным программированием, для запуска проекта из двух параллельно открытых терминалов можем исполнить:

    python telegram.py
    python app.py